In [2]:
import scipy
import pylab
import numpy as np
import matplotlib
from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from reproject import reproject_interp
from astropy.convolution import convolve,Gaussian2DKernel

#Embed plots in Jupyter window
%matplotlib inline

#Set plot fonts
matplotlib.rc('text', usetex=True)
matplotlib.rc('font', family='sans-serif')


In [22]:
#Reproject individual images to match the mosaic footprint in order to use them as masks
#Did regridding in CASA because I have bad habits

#dir = '/Users/hph/cmzoom_catalog/Continuum/'
dir='/Users/hph/Dropbox/CMZoom_Data/continuum_images/Final_continuum_images/Continuum_pbcor/'

#Some images missing-- can add them back in, reorder as desired, etc. 
images = pylab.array(['G1.683-0.089.continuum_regrid.fits',\ 
                      'G1.670-0.130.continuum_regrid.fits',\ 
                      'G1.651-0.050.continuum_regrid.fits',\ 
                      'G1.602+0.018.continuum_regrid.fits',\ 
                      'G1.085-0.027.continuum_regrid.fits',\ 
                      'G1.038-0.074.continuum_regrid.fits',\ 
                      'G0.891-0.048.continuum_regrid.fits',\
                      'G0.714-0.100.continuum_regrid.fits',\
                      'G0.699-0.028.continuum_regrid.fits',\
                      'G0.619+0.012.continuum_regrid.fits',\
                      'G0.489+0.010.continuum_regrid.fits',\
                      'G0.412+0.052.continuum_regrid.fits',\
                      'G0.380+0.050.continuum_regrid.fits',\
                      'G0.340+0.055.continuum_regrid.fits',\
                      'G0.326-0.085.continuum_regrid.fits',\
                      'G0.316-0.201.continuum_regrid.fits',\
                      'G0.253+0.016.continuum_regrid.fits',\
                      'G0.212-0.001.continuum_regrid.fits',\
                      'G0.145-0.086.continuum_regrid.fits',\
                      'G0.106-0.082.continuum_regrid.fits',\
                      'G0.070-0.035.continuum_regrid.fits',\
                      'G0.068-0.075.continuum_regrid.fits',\
                      'G0.054+0.027.continuum_regrid.fits',\
                      'G0.014+0.021.continuum_regrid.fits',\
                      'G0.001-0.058.continuum_regrid.fits',\
                      'G359.889-0.093.continuum_regrid.fits',\
                      'G359.865+0.022.continuum_regrid.fits',\
                      'G359.863-0.069.continuum_regrid.fits',\
                      'G359.734+0.002.continuum_regrid.fits',\
                      'G359.648-0.133.continuum_regrid.fits',\
                      'G359.611+0.018.continuum_regrid.fits',\
                      'G359.615-0.243.continuum_regrid.fits',\
                      'G359.484-0.132.continuum_regrid.fits',\
                      'G359.137+0.031.continuum_regrid.fits'])

names = pylab.array(['G1.683-0.089','G1.670-0.130','G1.651-0.050','G1.602+0.018',\
                    'G1.085-0.027','G1.038-0.074','G0.891-0.048','G0.714-0.100',\
                    'G0.699-0.028','G0.619+0.012','G0.489+0.010','G0.412+0.052',\
                    'G0.380+0.050','G0.340+0.055','G0.326-0.085','G0.316-0.201',\
                    'G0.253+0.016','G0.212-0.001','G0.145-0.086',\
                    'G0.106-0.082','G0.070-0.035','G0.068-0.075','G0.054+0.027',\
                    'G0.014+0.021','G0.001-0.058','G359.889-0.093','G359.865+0.022',\
                    'G359.863-0.069','G359.734+0.002','G359.648-0.133',\
                    'G359.611+0.018','G359.137+0.031','G359.484-0.132','G359.137+0.031'])                      

AKAs = pylab.array(['G1.6-a','G1.6-b','G1.6-c','G1.6-d','','','','','Sgr B2',\
                    'B2-West','e/f','d','c','b','Far-side 1','','Brick','','Straw',\
                    'Sticks','H2CO','Stone','Far-side 2','','50 km/s','20 km/s-a',\
                    'Far-side 3','20 km/s-b','Far-side 4','','Far-side 5','',\
                    'Sgr C',''])

#Define mask as the areas that are not NAN in the SMA images

Xs = pylab.zeros(35) #Arrays for mean pixel coordinates for each mask
Ys = pylab.zeros(35)

for i,image in enumerate(images):
    imap = fits.getdata(dir+image)
    maphead = fits.getheader(dir+image)
    
    if i==0:
        mask = imap.copy()*0.0+1000 #set background to 1000 (could be 0 if not using np.minimum below)
        masknan = np.isnan(mask)
        mask[masknan==1] = 1000
    
    mapnan = np.isnan(imap)
    imap[mapnan==1] = 1000 # background
    imap[mapnan==0] = i+1  # source number for the mask
    
    where = np.where(mapnan==0)
    Ys[i] = np.mean(where[0]) #save mean pixel coordinates
    Xs[i] = np.mean(where[1])
    
    mask = np.minimum(mask,map) #In cases where there are mask overlaps, go with the smaller-number mask
                                #This is arbitrary and mostly chosen to prioritize the Sgr B2 mask
        
hdu1 = fits.open('/Users/hph/cmzoom_catalog/Continuum/CMZoom_Mask_new.fits')[0]
fits.writeto('/Users/hph/cmzoom_catalog/Continuum/CMZoom_Mask_new.fits', mask,hdu1.header,overwrite=True)

ValueError: operands could not be broadcast together with shapes (497,934) (432,822) 